# Visualisations that will be used in the thesis


In [ ]:
import cairosvg
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D 
from matplotlib.patches import Patch
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path
import string
from io import BytesIO

import particles_vis_tooling as pv
import tooling
from env import PARTICLE_SIM_FOLDER, CONFIG_FOLDER, PARTICLE_PLOT_FOLDER, TEMPLATE_FOLDER, beaching_strats, coast_shapes, resolutions
from __init__ import logger

PLOT_FOLDER = Path("visualisations")
FORCE_PLOT = True
TEXT_WIDTH = 5.90666

In [ ]:
toml_fnames = [
    "flat_1km_naive.toml",
    "flat_1km_lebreton.toml",
    "flat_1km_oninkb.toml",
    "flat_1km_mheen.toml",
    "flat_1km_oninkbr.toml",
    "flat_2km_naive.toml",
    "flat_2km_lebreton.toml",
    "flat_2km_oninkb.toml",
    "flat_2km_mheen.toml",
    "flat_2km_oninkbr.toml",

    "concave_1km_naive.toml",
    "concave_1km_lebreton.toml",
    "concave_1km_oninkb.toml",
    "concave_1km_mheen.toml",
    "concave_1km_oninkbr.toml",

    "convex_1km_naive.toml",
    "convex_1km_lebreton.toml",
    "convex_1km_oninkb.toml",
    "convex_1km_mheen.toml",
    "convex_1km_oninkbr.toml",
]

def get_figsize(height=None, width=None, aspect_ratio=1.61803398875):
    """
    Gets the figure size for a given aspect ratio given the height or width of the figure.

    Aspect ratio is width/height, and defaults to the golden ratio. 
    """
    if height is None and width is None:
        raise ValueError("One of width or height must be provided.")
    
    if height is not None and width is not None:
        return width, height

    if height is not None:
        return height * aspect_ratio, height
    return width, width / aspect_ratio


In [ ]:
simulations = {fname: pv.ParticlesSimulation.from_toml(fname) for fname in toml_fnames}

# METHODOLOGY: `coastlines`


In [ ]:
fig_key = "coastlines"
plot_path = PLOT_FOLDER / f"{fig_key}.pdf"

def coast_to_bw(img):
    """
    Converts an NxMx3 image to an NxM image with values in [0, 1] representing black and white
    Converts white coastline to black, and blue water to white.
    """
    # Convert to greyscale
    img = np.sum(img, axis=2)
    mask = img == 3.0
    img[mask] = 0
    img[~mask] = 1
    return img
       
if not plot_path.exists() or FORCE_PLOT:
    coast_paths = {(code, name): TEMPLATE_FOLDER / "coastlines" / f"{name}.svg" for code, name in coast_shapes.items()}
    fig, axs = plt.subplots(1, 3, figsize=get_figsize(width = TEXT_WIDTH, aspect_ratio = 2.5), sharey=True)

    for ((code, name), path), ax in zip(coast_paths.items(), axs):
        img_png = cairosvg.svg2png(file_obj = open(path), output_width=120 * 6)
        img = mpl.image.imread(BytesIO(img_png))
        img = coast_to_bw(img)

        ax.imshow(img, cmap="gray", clim = (0,1), extent = (0, 120, 0, 126))
        ax.set(
            xlabel="x[km]",
            title=f"Coastline {code}",
        )

        start, end = ax.get_xlim()
        ax.xaxis.set_ticks(np.linspace(start, end, 5, endpoint=True))
        ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
        ax.yaxis.set_ticks([6, 40, 80, 126])



    handles = [
        Patch(facecolor="black", edgecolor="black", label="Coastline"),
        Patch(facecolor="white", edgecolor="black", label="Ocean")
        ]
    ax.legend(handles=handles)
    axs[0].set(ylabel="y[km]")
    for ax in axs:
        ax.grid(False)

    # fig.tight_layout()
    fig.savefig(plot_path, dpi=300)
    




# METHODOLOGY: `domain-regions`
Showing the concept of a beaching region


In [ ]:
"""
Plotting the land, beaching region and ocean to illustrate the different types of cells.
0 -> ocean
1 -> beaching region
2 -> land
"""
fig_key = "domain-regions"
plot_path = PLOT_FOLDER / f"{fig_key}.pdf"

if not plot_path.exists() or FORCE_PLOT:
    cmap_name = "Greys"
    sim_obj = simulations["convex_1km_oninkb.toml"]
    fig, ax = plt.subplots(figsize=get_figsize(width = TEXT_WIDTH))

    ds = sim_obj.ds_fieldset
    # Plotting landmask
    landmask = pv.process_landpoints_to_cells(ds.land.values) # True for cells that are land
    beaching_region = tooling.moore(landmask, 1) & ~landmask

    domain_values = np.zeros_like(landmask, dtype=int)
    domain_values[landmask] = 2
    domain_values[beaching_region] = 1

    X, Y = np.meshgrid(ds.x.values / 1_000, ds.y.values / 1_000) # Converting m to km
    ax.pcolor(X, Y, domain_values, cmap=cmap_name)

    # Custom legend
    cmap = mpl.cm.get_cmap(cmap_name)
    rgbs = cmap(np.linspace(0, 1, 3)) # Creating custom legend
    legend_names = ["Offshore region", "Beaching region", "Land"]
    handles = [Patch(facecolor=rgb, edgecolor="black", label=name) for rgb, name in zip(rgbs, legend_names)]
    ax.legend(handles=handles, loc = "upper left")

    # Custom gridlines in 1000m intervals

    # Setting minor gridlines
    zoom_window = [
        (20, 50), # xlim
        (0, 30), # ylim
    ]
    minor_ticks = [
        np.arange(zoom_window[0][0], zoom_window[0][1] + 1, 1),
        np.arange(zoom_window[1][0], zoom_window[1][1] + 1, 1),
    ]
    ax.set_xticks(minor_ticks[0], minor=True)
    ax.set_yticks(minor_ticks[1], minor=True)
    ax.grid(which="both", axis="both", alpha = 0.4, linestyle="-", linewidth=0.5)

    ax.set(
        xlim = zoom_window[0],
        ylim = zoom_window[1],
        xlabel = "x [km]",
        ylabel = "y [km]",
        aspect = "equal",
    )
    fig.tight_layout()
    fig.savefig(plot_path, dpi=300)

# Results: `currents`

In [ ]:
coasts = {
    "X": "flat_1km_oninkb.toml",
    "Y": "concave_1km_oninkb.toml",
    "Z": "convex_1km_oninkb.toml",
}
simulations_pstarts = {fname: pv.ParticlesSimulation.from_toml(fname, all_particles=True) for fname in ["flat_1km_oninkb.toml", "concave_1km_oninkb.toml", "convex_1km_oninkb.toml"]}

fig_key = "currents"
plot_path = PLOT_FOLDER / f"{fig_key}.png"

# rcparams context manager

if not plot_path.exists() or FORCE_PLOT:
    fig, axs = plt.subplots(1, 3, figsize=get_figsize(width=TEXT_WIDTH, aspect_ratio=15/8), sharex=True, sharey=True)
    for (coast_key, fname), ax in zip(coasts.items(), axs):
        sim_obj = simulations[fname]

        # Plotting quiver plot
        ds = sim_obj.ds_fieldset
        ax.streamplot(
            ds.x.values / 1_000, ds.y.values / 1_000, ds.U.values, ds.V.values,
            color = "#4B4B4B", density = 1.2, linewidth=0.4)#, alpha=0.5)

        # Speed
        U, V = ds.U.isel(x = slice(None, -1), y = slice(None, -1)).values, ds.V.isel(x = slice(None, -1), y = slice(None, -1)).values
        speed = np.sqrt(U**2 + V**2)
        X, Y = np.meshgrid(ds.x.values / 1_000, ds.y.values / 1_000) # Converting m to km
        c = ax.pcolor(X, Y, speed, clim = (0, 0.33), cmap = "viridis")

        # Plotting landmask
        landmask = pv.process_landpoints_to_cells(ds.land.values).astype(float) # Converting to float to mask out ocean
        landmask[landmask == 0] = np.nan
        ax.pcolor(X, Y, landmask, cmap="Greys", clim = (0, 1))

        ax.set(
            title=f"Coastline {coast_key}",
            xlabel = "x [km]",
            xlim = [0, 120],
            ylim = [0, 126],
            aspect="equal",
        )
        ax.xaxis.set_ticks([0, 40, 80, 120])
        
    cbar = fig.colorbar(c, ax=axs, location="bottom", shrink = 0.3)
    cbar.set_label("Speed [m/s]")
    axs[0].set_ylabel("y [km]")
    
    plt.savefig(plot_path, dpi=300)

# RESULTS: `particle-starts`
Particle initialisation points for coastlines X, Y, and Z at 1km resolution.

In [ ]:
fig_key = "particle-starts"
plot_path = PLOT_FOLDER / f"{fig_key}.png"

if not plot_path.exists() or FORCE_PLOT:

    fig, axs = plt.subplots(1, 3, figsize=get_figsize(width=5.9, aspect_ratio=15 / 8), sharex=True, sharey=True)
    for (coast_key, fname), ax in zip(coasts.items(), axs):
        sim_obj = simulations_pstarts[fname]

        # Plotting starting points of particles
        ds_trajectories = sim_obj.ds_trajectories.isel(obs = 0)[["lat", "lon", "ever_in_beaching_region"]]
        in_region = ds_trajectories.where(ds_trajectories.ever_in_beaching_region == 1, drop=True)
        out_region = ds_trajectories.where(ds_trajectories.ever_in_beaching_region == 0, drop=True)

        for temp_ds, color in zip([in_region, out_region], ["red", "#4B4B4B"]):
            ax.scatter(temp_ds.lon / 1_000, temp_ds.lat / 1_000, color = color, s = 0.2, marker=".") # Convert m to km

        # Plotting landmask
        ds = sim_obj.ds_fieldset
        landmask = pv.process_landpoints_to_cells(ds.land.values).astype(float) # Converting to float to mask out ocean
        landmask[landmask == 0] = np.nan
        X, Y = np.meshgrid(ds.x.values / 1_000, ds.y.values / 1_000) # Convert m to km
        ax.pcolor(X, Y, landmask, cmap="Greys", clim = (0, 1))

        ax.set(
            title=f"Coastline {coast_key}",
            xlabel = "x [km]",
            xlim = [0, 120],
            ylim = [0, 126],
            aspect="equal",
        )
        ax.xaxis.set_ticks([0, 40, 80, 120])
    axs[0].set_ylabel("y [km]")

    legend_elements = [
        Line2D([0], [0], color="red", marker='.', linestyle="", label='enters beaching region'),
        Line2D([0], [0], color="#4B4B4B", marker='.', linestyle="", label='never enters beaching region'),
    ]

    bbox = (0, -0.05, 1, 0.2)
    legend = fig.legend(handles=legend_elements, loc="upper center", bbox_to_anchor=bbox)

    fig.tight_layout()
    fig.savefig(plot_path, bbox_extra_artists=(legend,), dpi=300)

# Results: `currents-distribution`
Speed distributions of the flowfields

In [ ]:
fig, ax = plt.subplots(figsize=get_figsize(width=TEXT_WIDTH))
fig_key = "currents-distribution"
plot_path = PLOT_FOLDER / f"{fig_key}.pdf"
speeds = []
for index, (coast_key, fname) in enumerate(coasts.items()):
    sim_obj = simulations[fname]
    ds = sim_obj.ds_fieldset
    speed = np.sqrt(ds.U**2 + ds.V**2).values.flatten()

    # Filtering out zeros
    speed = speed[speed > 0]
    sns.kdeplot(speed, ax = ax, label = coast_key, color = f"C{index:02}")
    ax.axvline(speed.mean(), color = f"C{index:02}", linestyle = "--")
    logger.info(f"Mean speed for {coast_key}: {speed.mean():.3f} m/s")

ax.set(
    xlabel = "Current speed [m/s]",
    ylabel = "Density",
    xlim = [0, 0.4],
)
ax.minorticks_on()
ax.grid(which="minor", axis="both", linestyle="--", alpha = 0.5)
   
handles = [Patch(facecolor = f"C{i:02}", label = key) for i, key in enumerate(coasts.keys())] + [Line2D([], [], color="black", linestyle="--", label='mean')]
ax.legend(handles = handles)

fig.tight_layout()

if not plot_path.exists():
    fig.savefig(plot_path, dpi=300)

# RESULTS: `fig:particle-beached-timeseries`
(a) proportion of particles having entered the beaching region over time (b) proportion of particles beached over time. In these figures `particles' refers to the particles that enter the beaching region.

In [ ]:
fig_key = "particle-beached-timeseries"
plot_path = PLOT_FOLDER / f"{fig_key}.pdf"


def get_color(code):
    if len(code) == 1:
        beach = code
    else:
        beach = code[2]

    i = "ABCDE".index(beach)
    return f"C{i:02}"

def get_linestyle(code):
    if len(code) == 1:
        # Extract letter if full code is given
        coast = code
    else:
        coast = code[0]

    if coast == "X":
        return "-"
    elif coast == "Y":
        return "-."
    elif coast == "Z":
        return ":"

def get_marker(code):
    if len(code) == 1:
        # Extract letter if full code is given
        res = code
    else:
        res = code[1]
    
    if res == "2":
        return "."
    else:
        return ""

fig, axs = plt.subplots(2, 1, figsize=get_figsize(width=0.8*TEXT_WIDTH, aspect_ratio=0.8))
skip_sim = lambda sim_code: True if "2" in sim_code else False
for sim_obj in simulations.values():
    sim_code = sim_obj.sim_code
    if skip_sim(sim_code):
        continue
    ds = sim_obj.ds_trajectories

    if "A" in sim_code: # Just using A to represent general case
        # Prop in region
        days_beaching_region = ds.days_since_start.isel(obs = ds.beaching_region_obs.astype(int))
        days_beaching_region = np.sort(days_beaching_region.values)
        prop_in_region = np.arange(1, days_beaching_region.shape[0] + 1) / days_beaching_region.shape[0] # Accounting for 0 at t=0
        axs[0].plot([0] + list(days_beaching_region), [0] + list(prop_in_region), label = sim_code, color = "black", linestyle = get_linestyle(sim_code), marker = get_marker(sim_code))

    # Prop beached
    prop_beached = ds.beached.sum(dim="traj") / ds.traj.size
    days = prop_beached.days_since_start
    prop_beached = prop_beached.values
    axs[1].plot(days, prop_beached, label = sim_code, color = get_color(sim_code), linestyle = get_linestyle(sim_code), marker = get_marker(sim_code))



axs[0].set(
    xlabel = "Time [days]",
    ylabel = "Proportion crossed into beaching region",
)
axs[1].set(
    xlabel = "Time [days]",
    ylabel = "Proportion beached",
)
for ax in axs:
    ax.minorticks_on()
    ax.grid(which="minor", axis="both", linestyle="--", alpha = 0.5)

extra_artists = [axs[i].text(0, 1.01, f"\\textbf{{({string.ascii_lowercase[i]})}}", transform=axs[i].transAxes, size=mpl.rcParams["font.size"]) for i in range(len(axs))]

handles = [Line2D([], [], color="black", linestyle=get_linestyle(code), label=code) for code in "XYZ"]
legend1 = axs[1].legend(handles=handles, title="Coastline", loc="lower center")
fig.add_artist(legend1)
handles = [Patch(edgecolor="black", facecolor=get_color(code), label=code) for code in "ABCDE"]
legend2 = axs[1].legend(handles=handles, title="Beaching", loc="lower right")
fig.tight_layout()

if not plot_path.exists() or FORCE_PLOT:
    fig.savefig(plot_path, bbox_extra_artists=extra_artists+[legend1, legend2], dpi=300)

# APPPENDIX: Particle summary table
`app:appendix-particle-summary-table`
Summary table of results from the particle simulations

In [ ]:
df = pd.DataFrame.from_dict({key: pv.summarise_trajectories(sim_obj) for key, sim_obj in simulations.items()}, orient="index")
# Sort on column
df["Initial particles"] = 5000 # ? Manual override (as particles were dropped in the reading in of the file)
df = df.sort_values(by="Simulation code")
cols_to_drop = ["Particles exiting domain while having entered beaching region"] # Cols to drop when saved to tex
SAVE_TABLE = True
caption = (
    "Raw data from particle simulations. `Code' indicates the particle simulation. `Prop. in beaching region'\
 indicates the proportion of total particles that entered the beaching region. `Prop. beached' indicates the\
 proportion of particles beached out of the particles that entered the beaching region. `Crossings into' and\
 `Crossings out of' indicate the number of times particles crossed into and out of the beaching region.",
    "Raw particle summary data"
    )
if SAVE_TABLE:
    # df.drop(columns=cols_to_drop).to_csv(PLOT_FOLDER / "particle_trajectories_summary.csv")
    mapping = {
        "Simulation code": "Code",
        "Particles in beaching region": "No. in beaching region",
        "Proportion in beaching region": "Prop. in beaching region",
        "Particles beached": "No. beached",
        "Proportion beached": "Prop. beached",
        "Crossings into beaching region": "Crossings into",
        "Crossings out of beaching region": "Crossings out of",
    }
    df.rename(columns=mapping).drop(columns=cols_to_drop).to_latex(
        PLOT_FOLDER / "appendix-particle-summary-table.tex",
        index=False,
        caption=caption,
        float_format="{:0.3f}".format,
        )


# RESULTS: `particle-summary`
Summary bar charts of interesting stats

In [ ]:
fig_key = "particle-summary"
plot_path = PLOT_FOLDER / f"{fig_key}.pdf"

# Setting hatches
hatches = {
    "-": lambda code: "X" in code,
    "\\\\": lambda code: "Y" in code,
    "//": lambda code: "Z" in code,
}

def get_color_function(letter):
    """
    Doing this, as the following implementation didn't work as the lambdas shared the same memory address:
    ```
    colors = {f"C{i:02}": (lambda code: letter in code) for i, letter in enumerate("ABCDE")}
    ```
    """
    def func(code):
        return letter in code
    return func
colors = {f"C{i:02}": get_color_function(letter) for i, letter in enumerate("12")}

# rc params context manager
with plt.rc_context(rc={"font.size": 5}):
    fig, axs = plt.subplots(3, 2, figsize=get_figsize(width=TEXT_WIDTH, aspect_ratio=1/0.94))
    # ! Is it worth also plotting crossings into the beaching region
    cols_to_plot = np.array([
        ["Particles in beaching region", "Proportion in beaching region"],
        ["Particles beached", "Proportion beached"],
        ["Crossings into beaching region", "Crossings out of beaching region"],
    ])

    subfig_text = []
    for index, (col, ax) in enumerate(zip(cols_to_plot.flatten(), axs.flatten())):
        # Making barplot
        x = df["Simulation code"]
        y = df[col]
        patches = ax.bar(x=x, height=y, edgecolor="black")
        text = ax.text(0, 1.02, f"\\textbf{{({string.ascii_lowercase[index]})}}", transform=ax.transAxes, size=8)
        subfig_text.append(text)
        ax.set_xticklabels([i[2] for i in x]) # Extracting beaching letter
        # ax.axhline(5000, color="black", linestyle="--", label="Initial particles")

        ax.grid(False)
        ax.minorticks_on()
        ax.set_axisbelow(True)
        ax.grid(axis="y", which="both")

        for color, func in colors.items():
            for patch, code in zip(patches, x):
                if func(code):
                    patch.set_facecolor(color)
                    
        for hatch, func in hatches.items():
            for patch, code in zip(patches, x):
                if func(code):
                    patch.set_hatch(hatch)
        ax.set(
            ylabel = col,
        )

    # Configuring legend
    # bbox = (x0, y0, width, height)
    bbox = (0.52, 0)
    handles = [
        Patch(facecolor="white", edgecolor="black", label="X (flat)", hatch="-"),
        Patch(facecolor="white", edgecolor="black", label="Y (concave)", hatch="\\\\"),
        Patch(facecolor="white", edgecolor="black", label="Z (convex)", hatch="//"),
    ]
    legend1 = fig.legend(handles=handles, loc="upper right", title="Coastline", bbox_to_anchor=bbox)
    handles = [
        Patch(facecolor="C00", edgecolor="black", label="1 (1km)"),
        Patch(facecolor="C01", edgecolor="black", label="2 (2km)"),
    ]
    legend2 = fig.legend(handles=handles, loc="upper left", title="Resolution", bbox_to_anchor=bbox)
    fig.add_artist(legend1)
    fig.tight_layout()

    if not plot_path.exists() or FORCE_PLOT:
        plt.savefig(plot_path, bbox_extra_artists=(legend1, legend2) + tuple(subfig_text), bbox_inches='tight')

# RESULTS: `beaching-densities`

In [ ]:
fig_code = "beaching-densities"
plot_path = PLOT_FOLDER / f"{fig_code}.pdf"

fig, axs = plt.subplots(2, 3, figsize=get_figsize(width=TEXT_WIDTH, aspect_ratio=2), sharey="row", gridspec_kw = {'height_ratios':[3, 1]})
skip_sim = lambda sim_code: True if "2" in sim_code else False
for coast_code, ax in zip("XYZ", axs[0, :]):
    for sim_obj in simulations.values():
        sim_code = sim_obj.sim_code
        if skip_sim(sim_code):
            # Not doing 2km resolutions
            continue
        if coast_code not in sim_code:
            # Only doing specific coastline
            continue
        beached_lons = sim_obj.data.beached_lon_last

        sns.kdeplot(beached_lons / 1_000, label = sim_code, color = get_color(sim_code), linestyle = "-", ax=ax)

# Setting Axes options

for (coast_key, toml_fname), ax in zip(coasts.items(), axs[1:].flatten()):
    ds = simulations[toml_fname].ds_fieldset
    # Plotting landmask
    landmask = pv.process_landpoints_to_cells(ds.land.values).astype(float) # Converting to float to mask out ocean
    landmask[landmask == 0] = np.nan
    X, Y = np.meshgrid(ds.x.values / 1_000, ds.y.values / 1_000) # Converting m to km
    ax.pcolor(X, Y, landmask, cmap="Greys", clim = (0, 1))
    ax.set(
        ylim=(0, 40),
        aspect="equal",
    )

for ax, coast_code in zip(axs[0, :], "XYZ"):
    ax.minorticks_on()
    ax.grid(which="minor", axis="both", linestyle="--", alpha = 0.5)
    ax.set(
        ylabel="Density",
        xlim=(0, 120),
        title=f"Coastline {coast_code}",
    )

for ax in axs.flatten():
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(30))

axs[1, 0].set(
    ylabel="y[km]",
)
for ax in axs[1,:].flatten():
    ax.set(xlabel="x[km]")

artists = []
# handles = [Line2D([], [], color="black", linestyle=get_linestyle(code), label=code) for code in "XYZ"]
# artists.append(fig.legend(handles=handles, title="Coastline", loc="upper right"))
# fig.add_artist(legends[0])

handles = [Patch(edgecolor="black", facecolor=get_color(code), label=code) for code in "ABCDE"]
artists.append(fig.legend(handles=handles, title="Beaching", loc="center right"))

# Subfig text
# artists += [axs[i].text(0.01, 0.86, f"\\textbf{{({string.ascii_lowercase[i]})}}", transform=axs[i].transAxes, size=12) for i in range(len(axs))]

if not plot_path.exists() or FORCE_PLOT:
    fig.savefig(plot_path, bbox_extra_artists=artists, bbox_inches='tight')
fig.show()

# RESULTS: `particle-beached-timeseries-res`

In [ ]:
fig_key = "particle-beached-timeseries-res"
plot_path = PLOT_FOLDER / f"{fig_key}.pdf"

def get_linestyle_res(code):
    if len(code) == 1:
        # Extract letter if full code is given
        res = code
    else:
        res = code[1]

    
    if res == "2":
        return "--"
    else:
        return "-"


fig, ax = plt.subplots(figsize=get_figsize(width=0.75*TEXT_WIDTH))
# axs = axs.flatten()
skip_sim = lambda sim_code: True if "X" not in sim_code else False
for sim_obj in simulations.values():
    sim_code = sim_obj.sim_code
    if skip_sim(sim_code):
        continue
    ds = sim_obj.ds_trajectories

    # Prop beached
    prop_beached = ds.beached.sum(dim="traj") / ds.traj.size
    days = prop_beached.days_since_start
    prop_beached = prop_beached.values
    ax.plot(days, prop_beached, label = sim_code, color = get_color(sim_code), linestyle = get_linestyle_res(sim_code))



skip_sim = lambda sim_code: True if "2" in sim_code else False


ax.set(
    xlabel = "Time [days]",
    ylabel = "Proportion beached",
)

ax.minorticks_on()
ax.grid(which="minor", axis="both", linestyle="--", alpha = 0.5)

# extra_artists = [axs[i].text(0, 1.01, f"\\textbf{{({string.ascii_lowercase[i]})}}", transform=axs[i].transAxes, size=12) for i in range(len(axs))]
extra_artists = []

handles = [Line2D([], [], color="black", linestyle=get_linestyle_res(code), label=f"{code}km") for code in "12"]
legend1 = ax.legend(handles=handles, title="Resolution", loc="lower center")
fig.add_artist(legend1)
handles = [Patch(edgecolor="black", facecolor=get_color(code), label=code) for code in "ABCDE"]
legend2 = ax.legend(handles=handles, title="Beaching", loc="lower right")

if not plot_path.exists() or FORCE_PLOT:
    fig.tight_layout()
    fig.savefig(plot_path, bbox_extra_artists=extra_artists)

In [ ]:
fig_key = "beaching-densities-x-res"
plot_path = PLOT_FOLDER / f"{fig_key}.pdf"

fig, axs = plt.subplots(5, 1, sharex=True, figsize=get_figsize(width=TEXT_WIDTH, aspect_ratio=1.4))
axs_dict = {code: ax for code, ax in zip("ABCDE", axs)}

skip_sim = lambda sim_code: True if "X" not in sim_code else False
for sim_obj in simulations.values():
    sim_code = sim_obj.sim_code
    if skip_sim(sim_code):
        continue
    ds = sim_obj.ds_trajectories

    # Beached locations
    beached_lons = sim_obj.data.beached_lon_last
    sns.kdeplot(beached_lons / 1_000, label = sim_code, color = get_color(sim_code), linestyle = get_linestyle_res(sim_code), ax=axs_dict[sim_code[2]]) # Converting m to km


for ax in axs:
    ax.set(
        xlabel = "x [km]",
        xlim = (0, 120),
        ylabel = "Density",
    )
    ax.minorticks_on()
    ax.grid(which="minor", axis="both", linestyle="--", alpha = 0.5)

extra_artists = []
bbox = (1, 0.5)
handles = [Line2D([], [], color="black", linestyle=get_linestyle_res(code), label=f"{code}km") for code in "12"]
legend1 = fig.legend(handles=handles, title="Resolution", loc="lower right", bbox_to_anchor=bbox)
fig.add_artist(legend1)
handles = [Patch(edgecolor="black", facecolor=get_color(code), label=code) for code in "ABCDE"]
legend2 = fig.legend(handles=handles, title="Beaching", loc="upper right", bbox_to_anchor=bbox)
if not plot_path.exists() or FORCE_PLOT:
    fig.savefig(plot_path, bbox_extra_artists=extra_artists)



In [ ]:
fig_key = "beaching-densities-y-res"
plot_path = PLOT_FOLDER / f"{fig_key}.pdf"

fig, axs = plt.subplots(5, 1, sharex=True, figsize=get_figsize(width=TEXT_WIDTH, aspect_ratio=1.4))
axs_dict = {code: ax for code, ax in zip("ABCDE", axs)}

skip_sim = lambda sim_code: True if "X" not in sim_code else False
for sim_obj in simulations.values():
    sim_code = sim_obj.sim_code
    if skip_sim(sim_code):
        continue
    ds = sim_obj.ds_trajectories

    # Beached locations
    ds_last = sim_obj.ds_trajectories.isel(obs = -1)
    beached_lats = ds_last.where(ds_last.beached == 1).lat.values
    sns.kdeplot(beached_lats / 1000, label = sim_code, color = get_color(sim_code), linestyle = get_linestyle_res(sim_code), ax=axs_dict[sim_code[2]]) # Converting m to km


for ax in axs:
    ax.set(
        xlabel = "y [km]",
        xlim = (6, None),
        ylabel = "Density",
    )
    ax.minorticks_on()
    ax.grid(which="minor", axis="both", linestyle="--", alpha = 0.5)

extra_artists = []

handles = [Line2D([], [], color="black", linestyle=get_linestyle_res(code), label=f"{code}km") for code in "12"]
legend1 = fig.legend(handles=handles, title="Resolution", loc="center right")
fig.add_artist(legend1)
handles = [Patch(edgecolor="black", facecolor=get_color(code), label=code) for code in "ABCDE"]
legend2 = fig.legend(handles=handles, title="Beaching", loc="lower right")
if not plot_path.exists() or FORCE_PLOT:
    fig.savefig(plot_path, bbox_extra_artists=extra_artists)